# Initial RedisController design without redisworks

In [ ]:
class RedisController:
    """
    Manages dataframe or string retrievals and stores in Redis
    """
    def __init__(self, hostname, name):
        self.name = name
        self.redis = redis.StrictRedis.from_url(url=hostname)

    def get_df(self, name=None, key=None):
        self._validate_key(key)
        name = self._set_name(name)

        jsonified_df = self.redis.hget(name, key).decode("utf-8")
        df = pd.DataFrame(json.loads(jsonified_df))
        return df

    def get(self, name=None, key=None):
        self._validate_key(key)
        name = self._set_name(name)
        return self.redis.hget(name, key).decode("utf-8")

    def update(self, content, name=None, key=None, encoder=None):
        self._validate_key(key)
        name = self._set_name(name)

        # This JSON Encoder will handle things like numpy arrays and datetimes
        if not encoder:
            encoder = plotly.utils.PlotlyJSONEncoder

        if type(content) not in [pd.DataFrame, str]:
            raise TypeError('Not supported type')

        if type(content) == pd.DataFrame:
            content = json.dumps(content.to_dict(),
                                cls=encoder)

        self.redis.hset(name, key, content)

        return True

    def _set_name(self, name):
        if not name:
            name = self.name
        return name

    def _validate_key(self, key):
        if not key:
            raise Exception('Need key')

# Play around with redisworks!

In [68]:
from redisworks import Root
import datetime

In [69]:
root = Root()

In [ ]:
root.time = datetime.datetime.now()
root.the.mapping.example = {1: 1, "a": "b"}

In [79]:
root.my.list = [[1, 3, 4]]

In [80]:
root.flush()

In [84]:
print(root.time)

2019-07-30 11:11:08.702255


In [86]:
time = root.time
print(type(time))

<class 'dot.dot.LazyDot'>


In [82]:
print(type(root.my.list))

<class 'dot.dot.LazyDot'>


In [5]:
print(root.the.mapping.example)

{1: 1, 'a': 'b'}


# New RedisController design with redisworks

In [91]:
import datetime
import json
import pandas as pd
import plotly
import redisworks

class RedisInstance(redisworks.Root):
    def delete(self, item):
        item = "%s.%s" % (self._registry.root_name, item)
        self.red.delete(item)

class RedisHandler:
    def __init__(self, name, key):
        self.name = name
        self.key = key

        self.base_path = f'{self.name}.{self.key}'
        self.table = RedisInstance()

    def update(self, content, encoder=None):

        # This JSON Encoder will handle things like numpy arrays and datetimes
        if not encoder:
            encoder = plotly.utils.PlotlyJSONEncoder

        content_json = content

        if type(content) == pd.DataFrame:
            content_json = json.dumps(content.to_dict(),
                                cls=encoder)

            self.table[self._get_path('df_types')] = [[str(dtype) for dtype in content.dtypes]]

        self.table[self._get_path('updated_time')] = datetime.datetime.now()
        self.table[self._get_path('content')] = content_json
        self.table.flush()

    def get(self):
        base_path = f'{self.name}.{self.key}'

        content = self.table[self._get_path('content')]
        updated_time = self.table[self._get_path('updated_time')]
        df_types = self.table[self._get_path('df_types')]

        if not content and not updated_time:
            return None

        # Due to redisworks list storing behavior
        if df_types:
            df_types = df_types[0]

        buffer = dict(content=content,
                      updated_time=updated_time,
                      df_types=df_types)

        return buffer

    def delete(self):
        self.table.delete(self._get_path('updated_time'))
        self.table.delete(self._get_path('content'))
        return True

    def _get_path(self, key):
        return f'{self.base_path}.{key}'

class RedisController:
    """
    Manages dataframe or string retrievals and stores in Redis
    """
    def __init__(self, hostname, name):
        self.name = name

    def get_df(self, name=None, key=None):
        self._validate_key(key)
        name = self._set_name(name)

        handler = RedisHandler(name, key)
        data = handler.get()
        print(data['content'][:30])
        print(type(data['content']))

        data['content'] = str(data['content'])
        df = pd.DataFrame(json.loads(data['content']))

        DATETIME_REGEX = '(datetime)'

        # If key df_types exist, retrieved data must be a dataframe
        # If yes, cast time columns by found time datatype columns
        if data['df_types']:
            for idx, dtype in enumerate(data['df_types']):
                if re.search(DATETIME_REGEX, dtype):
                    df.iloc[:,idx] = df.iloc[:,idx].astype(dtype)
        return df

    def get(self, name=None, key=None):
        self._validate_key(key)
        name = self._set_name(name)
        handler = RedisHandler(name, key)
        return handler.get()

    def update(self, content, name=None, key=None, encoder=None):
        self._validate_key(key)
        name = self._set_name(name)

        if type(content) == list:
            raise TypeError(f'{type(content)} not supported')

        handler = RedisHandler(name, key)
        handler.update(content, encoder=encoder)

        return True

    def delete(self, name=None, key=None):
        self._validate_key(key)
        name = self._set_name(name)
        handler = RedisHandler(name, key)
        return handler.delete()

    def _set_name(self, name):
        if not name:
            name = self.name
        return name

    def _validate_key(self, key):
        if not key:
            raise Exception('Need key')


In [92]:
import os
REDIS_URL = os.environ.get('REDIS_URL', 'redis://localhost:6379')
REDIS_HASH_NAME = os.environ.get("DASH_APP_NAME", "app_data")
redis_instance = RedisController(hostname=REDIS_URL, name=REDIS_HASH_NAME)